In [8]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 242 kB 859 kB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8917 sha256=7e80a16ba49dda5cd186f7e442483597812fc0e7bd7af88a77aa8d65e9b47632
  Stored in directory: /home/work/.cache/pip/wheels/ef/92/e0/956c00decf3091294b8800d9104f6169a1adacfaafd122ed42
Successfully built et-xmlfile
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [10]:
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 103 kB 498 kB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [71]:
import openpyxl
import pandas as pd

xlsxFile = '식수(19.01_20.10).xlsx'
sheetList = []

wb = openpyxl.load_workbook(xlsxFile)
for i in wb.sheetnames:
    sheetList.append(i)

import pandas as pd
ppl=[]
for i in range(len(sheetList)):
    if i%2==0:
        ppl.append(pd.read_excel("식수(19.01_20.10).xlsx",index_col=0,sheet_name=sheetList[i]).iloc[:32,0:4])
ppl    

[                    Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4
 1월 식 수 정 산 (직원+외부)                                             
 날짜                          야식         조식         중식         석식
 2019-01-01 00:00:00         32         12         92         41
 2019-01-02 00:00:00         33         13        164         47
 2019-01-03 00:00:00         35         10        153         46
 2019-01-04 00:00:00         35         16        153         45
 2019-01-05 00:00:00         33          9         52         28
 2019-01-06 00:00:00         10         10         26         26
 2019-01-07 00:00:00         15         11        156         47
 2019-01-08 00:00:00         33         15        153         54
 2019-01-09 00:00:00         33         15        160         42
 2019-01-10 00:00:00         35         16        164         50
 2019-01-11 00:00:00         32         13        144         55
 2019-01-12 00:00:00         26         16         50         21
 2019-01-13 00:00:00     

In [72]:
for i in range(len(ppl)-1):
    ppl[i].columns=ppl[i].iloc[0,:]
    ppl[i]=ppl[i].iloc[1:,:]

In [73]:
for i in range(1,len(ppl)):
    ppl[0]=pd.concat([ppl[0],ppl[i]])
ppl[0]

날짜,야식,조식,중식,석식
2019-01-01,32,12,92,41
2019-01-02,33,13,164,47
2019-01-03,35,10,153,46
2019-01-04,35,16,153,45
2019-01-05,33,9,52,28
...,...,...,...,...
2020-10-27,36,20,162,55
2020-10-28,34,19,159,50
2020-10-29,33,16,160,50
2020-10-30,35,20,150,50


In [76]:
ppl[0]=ppl[0].reset_index()

In [88]:
ppl[0]=ppl[0].drop(ppl[0][ppl[0]['index']=='합계'].index.values)

In [99]:
data=ppl[0]

In [84]:
data=pd.read_csv('횡성 input1.csv')

In [85]:
data['timestamp']=data['timestamp'].apply(lambda x:pd.to_datetime(x))
# data['dayofweek_mean']='.'
# data['dayofweek_median']='.'
# import calendar
# data['weekday']=data['timestamp'].apply(lambda x: calendar.weekday(x.year,x.month,x.day))

In [21]:
data.groupby(['weekday']).mean()

,target_value,day_sin,dayofweek_median,abnormal,holiday,before_holiday,after_holiday
weekday,,,,,,,
0,300.589474,7.818315e-01,310.0,0.410526,0.042105,0.010526,0.021053
1,313.093750,9.749279e-01,319.0,0.416667,0.031250,0.052083,0.031250
2,307.385417,4.338837e-01,317.0,0.406250,0.062500,0.052083,0.020833
3,309.947917,-4.338837e-01,317.0,0.395833,0.062500,0.031250,0.052083
4,284.229167,-9.749279e-01,301.0,0.406250,0.052083,0.020833,0.041667
5,64.536082,-7.818315e-01,59.0,0.412371,0.051546,0.041237,0.020619
6,13.905263,-2.450000e-16,13.0,0.400000,0.042105,0.021053,0.042105


In [56]:
data.groupby(['weekday']).median()
def func(total):
    if total['weekday']==0:
        return 310
    elif total['weekday']==1: 
        return 319
    elif total['weekday']==2: 
        return 317
    elif total['weekday']==3: 
        return 317
    elif total['weekday']==4: 
        return 301
    elif total['weekday']==5: 
        return 59
    elif total['weekday']==6: 
        return 13
    else:
        return 0
        pass 
    
data.dayofweek_median=data.apply(func,axis=1)

In [57]:
data.groupby(['weekday']).mean()
def func(total):
    if total['weekday']==0:
        return 300
    elif total['weekday']==1: 
        return 313
    elif total['weekday']==2: 
        return 307
    elif total['weekday']==3: 
        return 309
    elif total['weekday']==4: 
        return 284
    elif total['weekday']==5: 
        return 64
    elif total['weekday']==6: 
        return 13
    else:
        return 0
        pass 
    
data.dayofweek_mean=data.apply(func,axis=1)

In [86]:
# 과거 temperature    
from bs4 import BeautifulSoup
import requests
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy={year}&obs=08&x=32&y=9'.format(year='2018'))  #천안 232 횡성 114 서울 108
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy={year}&obs=08&x=32&y=9'.format(year='2019'))  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy={year}&obs=08&x=32&y=9'.format(year='2020'))  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['temperature']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))

In [87]:
import numpy as np
def abnormal(prev):
    try: 
        if (prev['timestamp'].month in [6,7,8]) & (float(prev['temperature'])>=28):
            return 1
        elif (prev['timestamp'].month in [9,10,11]) & (float(prev['temperature'])>=5):
            return 1
        elif (prev['timestamp'].month in [12,1,2]) & (float(prev['temperature'])<=-5):
            return 1
        elif (prev['timestamp'].month in [3,4,5]) & (float(prev['temperature'])<=5):
            return 1
        else:
            return 0
    except:
        prev['temperature']==np.nan
            
data['abnormal']=data.apply(abnormal,axis=1) 

In [88]:
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy=2018&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy=2019&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=114&yy=2020&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['rainfall']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))

In [52]:
holiday=[]
date=['01','02','03','04','05','06','07','08','09','10','11','12']
for j in range(2019,2021):
    for i in range(len(date)):
        url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str(j),date=date[i])
        resp=requests.get(url)
        soup=BeautifulSoup(resp.content)
        Text=soup.find_all('locdate')
        for i in range(len(Text)):
            holiday.append(pd.to_datetime(Text[i].text)) 
            
def holiday1(x):
    if (x in holiday)==False:
        return 0
    else:
        return 1
    
from datetime import timedelta    
def before_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=1) in holiday:
        return 1
    else:
        return 0    

def after_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=-1) in holiday:
        return 1
    else:
        return 0 
        
data['holiday']=data['timestamp'].apply(lambda x: holiday1(x))
data['before_holiday']=data.apply(before_holiday,axis=1)
data['after_holiday']=data.apply(after_holiday,axis=1)   

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [89]:
data['rainfall']=data['rainfall'].apply(lambda x: float(x.replace('\xa0','0')))
data.to_csv('횡성input.csv',index=False)

In [1]:
import openpyxl
import pandas as pd
import pandas as pd
present=pd.read_excel("본사_천안_횡성_김천_출근 내역(2018_2020).xlsx",index_col=0,sheet_name='천안') #.iloc[:32,0:4])    

In [2]:
present=present.reset_index()

In [4]:
present=present[(present['WORK_DATE']>='2019-01-01')&(present['WORK_DATE']<='2020-10-31')]

In [5]:
present=present.rename(columns={'WORK_DATE':'timestamp','인원 수':'gate'})

In [39]:
present=present.reset_index().drop('index',axis=1)

In [65]:
data=data.reset_index().drop('index',axis=1)

In [67]:
data

,timestamp,target_value,weekday,day_sin,dayofweek_mean,dayofweek_median,temperature,abnormal,rainfall,holiday,before_holiday,after_holiday
0,2019-01-01,107,1,0.974928,313,319,-0.6,0,,1,0,0
1,2019-01-02,306,2,0.433884,307,317,0.2,0,,0,0,1
2,2019-01-03,299,3,-0.433884,309,317,3.2,0,,0,0,0
3,2019-01-04,308,4,-0.974928,284,301,4.1,0,,0,0,0
4,2019-01-05,56,5,-0.781831,64,59,1.1,0,,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
665,2020-10-27,285,1,0.974928,313,319,19.4,1,,0,0,0
666,2020-10-28,280,2,0.433884,307,317,18.2,1,,0,0,0
667,2020-10-29,315,3,-0.433884,309,317,16.1,1,,0,0,0
668,2020-10-30,187,4,-0.974928,284,301,19.6,1,,0,0,0


In [7]:
present.drop('timestamp',axis=1)

,gate
0,94
1,219
2,229
3,211
4,48
...,...
665,226
666,229
667,216
668,140


In [68]:
data=pd.concat([data,present.drop('timestamp',axis=1)],axis=1)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data=pd.read_csv('횡성 input.csv')
y=data[['중식']]
X=data[data.columns.difference(['timestamp','중식','석식','야식','조식'])]
X['rainfall']=X['rainfall'].apply(lambda x: float(x.replace('\xa0','0')))
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(X)
y=scaler.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(603, 10)
(67, 10)
(603, 1)
(67, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [2]:
X_train=X_train.reshape((-1,1,10))
X_test=X_test.reshape((-1,1,10))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(603, 1, 10)
(67, 1, 10)
(603, 1)
(67, 1)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, Dense
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Activation
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (1,10)))
model.add(Dense(1))
model.add(Activation('softmax'))
adam = optimizers.Adam(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 

# model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 2)
model.fit(X_train, y_train,epochs = 200, batch_size = 50, verbose = 2)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_)) 
RMSE = mean_squared_error(y_test, y_pred)**0.5
RMSE

Epoch 1/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 2/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 3/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 4/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 5/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 6/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 7/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 8/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 9/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 10/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 11/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 12/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 13/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 14/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 15/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 16/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 17/200
13/1

0.5672534841560553

In [ ]:
pd.read('input2_daysi')